In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from pathlib import Path

import pytorch_lightning as pl
import torch
import wandb
from sdofm import utils
from sdofm.datasets import SDOMLDataModule, DimmedSDOMLDataModule
from sdofm.pretraining import MAE
from sdofm.finetuning import Autocalibration

In [5]:
import omegaconf
cfg = omegaconf.OmegaConf.load('../experiments/default.yaml')

In [11]:
data_module = SDOMLDataModule(
    hmi_path=None,
    aia_path=os.path.join(
        cfg.data.sdoml.base_directory, cfg.data.sdoml.sub_directory.aia
    ),
    eve_path=None,
    components=cfg.data.sdoml.components,
    wavelengths=cfg.data.sdoml.wavelengths,
    ions=cfg.data.sdoml.ions,
    frequency=cfg.data.sdoml.frequency,
    batch_size=cfg.model.opt.batch_size,
    num_workers=cfg.data.num_workers,
    val_months=cfg.data.month_splits.val,
    test_months=cfg.data.month_splits.test,
    holdout_months=cfg.data.month_splits.holdout,
    cache_dir=os.path.join(
        cfg.data.sdoml.base_directory, cfg.data.sdoml.sub_directory.cache
    )
)
data_module.setup()

[* CACHE SYSTEM *] Found cached index data in /mnt/sdoml/cache/aligndata_AIA_FULL_12min.csv.
[* CACHE SYSTEM *] Found cached normalization data in /mnt/sdoml/cache/normalizations_AIA_FULL_12min.json.


In [12]:
dimmed_data_module = DimmedSDOMLDataModule(
    
    hmi_path=None,
    aia_path=os.path.join(
        cfg.data.sdoml.base_directory, cfg.data.sdoml.sub_directory.aia
    ),
    eve_path=None,
    components=cfg.data.sdoml.components,
    wavelengths=cfg.data.sdoml.wavelengths,
    ions=cfg.data.sdoml.ions,
    frequency=cfg.data.sdoml.frequency,
    batch_size=cfg.model.opt.batch_size,
    num_workers=cfg.data.num_workers,
    val_months=cfg.data.month_splits.val,
    test_months=cfg.data.month_splits.test,
    holdout_months=cfg.data.month_splits.holdout,
    cache_dir=os.path.join(
        cfg.data.sdoml.base_directory, cfg.data.sdoml.sub_directory.cache
    )
)
dimmed_data_module.setup()

[* CACHE SYSTEM *] Found cached index data in /mnt/sdoml/cache/aligndata_AIA_FULL_12min.csv.
[* CACHE SYSTEM *] Found cached normalization data in /mnt/sdoml/cache/normalizations_AIA_FULL_12min.json.


In [13]:
model = MAE(
        **cfg.model.mae,
        optimiser=cfg.model.opt.optimiser,
        lr=cfg.model.opt.learning_rate,
        weight_decay=cfg.model.opt.weight_decay,
)

In [16]:
trainer = pl.Trainer(devices=2, accelerator=cfg.experiment.accelerator, max_epochs=cfg.model.opt.epochs)
trainer.fit(model=model, datamodule=data_module)

MisconfigurationException: No supported gpu backend found!

In [49]:
model = Autocalibration(
        **cfg.model.mae,
        **cfg.model.dimming,
        optimiser=cfg.model.opt.optimiser,
        lr=cfg.model.opt.learning_rate,
        weight_decay=cfg.model.opt.weight_decay,
)

In [20]:
dl = data_module.train_dataloader()

In [27]:
data_module.aligndata

,idx_131A,idx_1600A,idx_1700A,idx_171A,idx_193A,idx_211A,idx_304A,idx_335A,idx_94A,idx_Bx,idx_eve
Time,,,,,,,,,,,
2014-05-26 08:00:00,9071,9073,8189,9059,9069,9071,9069,9068,9072,37069,2136414
2014-05-26 08:12:00,9073,9075,8190,9061,9071,9073,9071,9070,9074,37124,2136426
2014-05-26 08:24:00,9075,9077,8192,9063,9073,9075,9073,9072,9076,37133,2136438
2014-05-26 08:36:00,9077,9079,8194,9065,9075,9077,9075,9074,9078,37066,2136450
2014-05-26 08:48:00,9079,9081,8196,9067,9077,9079,9077,9076,9080,37098,2136462
2014-05-26 09:00:00,9081,9083,8198,9069,9079,9081,9079,9078,9082,37042,2136474
2014-05-26 09:12:00,9083,9085,8199,9071,9081,9083,9081,9080,9084,37047,2136486
2014-05-26 09:24:00,9085,9087,8201,9073,9083,9085,9083,9082,9086,37120,2136498
2014-05-26 09:36:00,9087,9089,8203,9075,9085,9087,9085,9084,9088,37031,2136510
